In [ ]:
import numpy as np
import pandas as pd
import bisect
from numpy import datetime64
import warnings

warnings.filterwarnings('ignore')

In [ ]:
def get_season(date: datetime64) -> int:
    x = date.month
    if x == 12:
        return 1
    return bisect.bisect_left([1, 6, 8, 11], x) + 1

In [ ]:
df = pd.read_csv('files\\main_kgs.csv')
df

In [ ]:
df.dtypes

In [ ]:
keys = df['obj_key'].unique()

In [ ]:
df['ДатаначалаБП0'].fillna(df['ДатаНачалаЗадачи'], inplace=True)
df['ДатаокончанияБП0'].fillna(df['ДатаОкончанияЗадачи'], inplace=True)
df

In [ ]:
exs = df['Экспертиза'].unique()

d_exs = {exs[i]: i for i in range(len(exs))}
d_exs

In [ ]:
df['Статуспоэкспертизе'].fillna(0, inplace=True)
df['Экспертиза'] = df['Экспертиза'].map(d_exs)
df

In [ ]:
df['Статуспоэкспертизе'] = df['Статуспоэкспертизе'].astype(int)

In [ ]:
df['ПроцентЗавершенияЗадачи'].fillna(0, inplace=True)
df['ПроцентЗавершенияЗадачи'] = df['ПроцентЗавершенияЗадачи'].astype(int)
df

In [ ]:
atts = pd.read_csv("files\\attributes.csv")
atts['date_report'] = pd.to_datetime(atts['date_report'])

In [ ]:
len(df)

In [ ]:
df.drop_duplicates(inplace=True)
len(df)

In [ ]:
df['ДатаНачалаЗадачи'] = pd.to_datetime(df['ДатаНачалаЗадачи'])
df['ДатаОкончанияЗадачи'] = pd.to_datetime(df['ДатаОкончанияЗадачи'])
df['ДатаначалаБП0'] = pd.to_datetime(df['ДатаначалаБП0'])
df['ДатаокончанияБП0'] = pd.to_datetime(df['ДатаокончанияБП0'])
df['date_report'] = pd.to_datetime(df['date_report'])

In [ ]:
main_keys = ['1', '1.4.2', '3', '3.1', '3.3', '3.8', '3.11', '3.13', '3.15', '4', '4.1.1', '4.2', '4.3', '4.4', '4.5',
             '4.6', '4.7', '4.8', '4.9', '4.10', '4.11', '4.12', '5', '5.11', '7.1', '7.1.8', '7.4', '8']
df

In [ ]:
df = df[df['Кодзадачи'].isin(main_keys)]

In [ ]:
df['season'] = df['ДатаНачалаЗадачи'].apply(get_season)
df

In [ ]:
columns = ['obj_key', 'Кодзадачи', "ДатаНачалаЗадачи", "ДатаОкончания задачи", 'ПроцентЗавершенияЗадачи', 'obj_prg',
           'obj_subprg', 'ДатаначалаБП0', 'ДатаокончанияБП0', 'Статуспоэкспертизе', 'Экспертиза',
           'season', "Сдвиг", ]

# Create an empty DataFrame with columns
df2 = pd.DataFrame(columns=columns)
from tqdm import tqdm

# Iterate over rows and append data
for key in tqdm(keys):
    codes = df[df['obj_key'] == key]['Кодзадачи'].unique()
    codes.sort()
    for code in codes:
        result = df[(df['obj_key'] == key) & (df['Кодзадачи'] == code)].sort_values(
            ['ПроцентЗавершенияЗадачи', 'date_report'])
        if not result.empty:
            first_line = result.head(1)
            last_line = result.tail(1)

            start_date = first_line['date_report'].values[0]
            end_date = last_line['date_report'].values[0]
            task_ready = last_line['ПроцентЗавершенияЗадачи'].values[0]
            days_difference = (end_date - start_date) / np.timedelta64(1, 'D')
            obj_prg = first_line['obj_prg'].values[0]
            obj_subprg = first_line['obj_subprg'].values[0]
            start_bpo = first_line['ДатаначалаБП0'].values[0]
            end_bpo = last_line['ДатаокончанияБП0'].values[0]
            status_exp = last_line['Статуспоэкспертизе'].values[0]
            exp = last_line['Экспертиза'].values[0]
            season = first_line['season'].values[0]
            row_data = [key, code, start_date, end_date, task_ready, obj_prg, obj_subprg, start_bpo, end_bpo, status_exp, exp, season, abs(days_difference)] # abs(days_difference)
            df2 = df2.append(pd.Series(row_data, index=columns), ignore_index=True)
        else:
            print(key, code)
# Print the resulting DataFrame
df2

In [ ]:
df2['Сдвиг'].replace({0.0: 1}, inplace=True)
df2['Сдвиг'] = df2['Сдвиг'].astype(int)
df2['Скорость'] = df2['ПроцентЗавершенияЗадачи'] / df2['Сдвиг']
df2

In [ ]:
df['Скорость'] = 0
keys = df['obj_key'].unique()
for key in tqdm(keys):
    codes = df[df['obj_key'] == key]['Кодзадачи'].unique()
    codes.sort()
    for code in codes:
        speed = df2[(df2['obj_key'] == key) & (df2['Кодзадачи'] == code)].head(1)['Скорость'].values[0]
        df.loc[(df['obj_key'] == key) & (df['Кодзадачи'] == code), 'Скорость'] = speed
df

In [ ]:
atts['Площадь'].fillna(0, inplace=True)
atts['Кол-во рабочих'].fillna(0, inplace=True)
atts

In [ ]:
df['Кол-во рабочих'] = np.nan
df['Генподрядчик'] = np.nan
df['Генпроектировщик'] = np.nan

In [ ]:
for key in tqdm(keys):
    dates = atts[atts['obj_key'] == key]['date_report'].unique()
    dates.sort()
    for date in dates:
        query = atts[(atts['obj_key'] == key) & (atts['date_report'] == date)]
        if not query.empty:
            rab = query.head(1)['Кол-во рабочих'].values[0]
            gen_pod = query.head(1)['Генподрядчик'].values[0]
            gen_proc = query.head(1)['Генпроектировщик'].values[0]
            mask = (df['obj_key'] == key) & (df['date_report'] == date)
            df.loc[mask, 'Кол-во рабочих'] = rab
            df.loc[mask, 'Генподрядчик'] = gen_pod
            df.loc[mask, 'Генпроектировщик'] = gen_proc
        else:
            print("skip")
df

In [ ]:
df['Кол-во рабочих'].value_counts()

In [ ]:
df[df['obj_key'] == '022-0170']

In [ ]:
len(df)

In [ ]:
obj_prg_ex = df['obj_prg'].unique()
obj_subprg_ex = df['obj_subprg'].unique()
obj_subprg_ex_d = {obj_subprg_ex[i]: i for i in range(len(obj_subprg_ex))}
obj_prg_ex_d = {obj_prg_ex[i]: i for i in range(len(obj_prg_ex))}
df['obj_prg'] = df['obj_prg'].map(obj_prg_ex_d)
df['obj_subprg'] = df['obj_subprg'].map(obj_subprg_ex_d)

In [ ]:
df.drop(['ДатаначалаБП0', 'ДатаокончанияБП0'], axis=1, inplace=True)
df

In [ ]:
df['Время на выполнение'] = abs((df['ДатаНачалаЗадачи'] - df['ДатаОкончанияЗадачи']).dt.days)
df

In [ ]:
obj_prg_ex_d

In [ ]:
df.dtypes

In [ ]:
df.sort_values(['obj_key', 'Кодзадачи', 'ПроцентЗавершенияЗадачи', 'date_report']).values

In [ ]:
df[df['obj_key']== '022-0170'].sort_values(['obj_key', 'Кодзадачи', 'ПроцентЗавершенияЗадачи', 'date_report'])

In [ ]:
tst_df = df.loc[df['Кол-во рабочих'].isna()].sort_values(['obj_key', 'Кодзадачи', 'ПроцентЗавершенияЗадачи', 'date_report'])
for i, row in tqdm(tst_df.iterrows()):
    obj_key = row[2]
    date_df = row[9]
    dates = atts[atts['obj_key'] == obj_key]['date_report'].unique()
    min_date = min(dates, key=lambda x: abs(x - date_df))
    query = atts[(atts['obj_key'] == obj_key) & (atts['date_report'] == min_date)].head(1)
    gen_pod = query['Генподрядчик'].values[0]
    gen_proc = query['Генпроектировщик'].values[0]
    rab = query['Кол-во рабочих'].values[0]
    square = query['Площадь'].values[0]
    tst_df.at[i, 'Генподрядчик'] = gen_pod
    tst_df.at[i, 'Генпроектировщик'] = gen_proc
    tst_df.at[i, 'Кол-во рабочих'] = rab
    tst_df.at[i, 'Площадь'] = square
tst_df

In [261]:
df.update(tst_df[['Генподрядчик', 'Генпроектировщик', 'Кол-во рабочих', 'Площадь']])
df

,obj_prg,obj_subprg,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,Статуспоэкспертизе,Экспертиза,date_report,season,Скорость,Кол-во рабочих,Генподрядчик,Генпроектировщик,Время на выполнение
0,0,0,022-0527,1,0,2022-01-14,2023-03-30,0,0,2023-01-17,1,0.257576,0.0,0.0,0.0,440
1,0,0,022-0354,1,0,2022-12-01,2023-07-27,0,0,2023-01-17,1,0.462121,0.0,0.0,0.0,238
2,0,1,022-0513,1,0,2023-01-23,2023-09-29,0,0,2023-01-17,1,0.416667,0.0,0.0,0.0,249
3,0,1,020-0684,1,0,2020-11-03,2022-02-01,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,455
4,0,1,019-0589,1,0,2020-11-03,2022-05-16,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055395,1,2,019-0477,7.4,93,2020-04-28,2023-05-31,0,0,2023-05-29,2,0.704545,5.0,12.0,16.0,1128
1056093,0,0,020-0710,8,0,2023-08-31,2024-04-10,0,0,2023-05-29,3,0.000000,156.0,5.0,8.0,223
1056094,0,0,019-0675,8,0,2023-09-01,2024-04-11,0,0,2023-05-29,4,0.000000,139.0,22.0,33.0,223
1056095,0,1,020-0712,8,0,2023-06-19,2024-01-26,0,0,2023-05-29,2,0.000000,53.0,5.0,8.0,221


In [263]:
df['Площадь'] = tst_df['Площадь']
df['Площадь'].fillna(0, inplace=True)
df

,obj_prg,obj_subprg,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,Статуспоэкспертизе,Экспертиза,date_report,season,Скорость,Кол-во рабочих,Генподрядчик,Генпроектировщик,Время на выполнение,Площадь
0,0,0,022-0527,1,0,2022-01-14,2023-03-30,0,0,2023-01-17,1,0.257576,0.0,0.0,0.0,440,9850.0
1,0,0,022-0354,1,0,2022-12-01,2023-07-27,0,0,2023-01-17,1,0.462121,0.0,0.0,0.0,238,20271.0
2,0,1,022-0513,1,0,2023-01-23,2023-09-29,0,0,2023-01-17,1,0.416667,0.0,0.0,0.0,249,6892.0
3,0,1,020-0684,1,0,2020-11-03,2022-02-01,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,455,2488.6
4,0,1,019-0589,1,0,2020-11-03,2022-05-16,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,559,3751.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055395,1,2,019-0477,7.4,93,2020-04-28,2023-05-31,0,0,2023-05-29,2,0.704545,5.0,12.0,16.0,1128,0.0
1056093,0,0,020-0710,8,0,2023-08-31,2024-04-10,0,0,2023-05-29,3,0.000000,156.0,5.0,8.0,223,0.0
1056094,0,0,019-0675,8,0,2023-09-01,2024-04-11,0,0,2023-05-29,4,0.000000,139.0,22.0,33.0,223,0.0
1056095,0,1,020-0712,8,0,2023-06-19,2024-01-26,0,0,2023-05-29,2,0.000000,53.0,5.0,8.0,221,0.0


In [264]:
df.to_csv("files\\pred_final.csv", index=False)